<a href="https://colab.research.google.com/github/baerenstein/qFinance/blob/main/Research/ESG%20High%20Yield.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Portfolio Analysis - DWS ESG Euro High Yield

## Introduction
In this notebook I replicate and analyse the [DWS Invest ESG Euro High Yield](https://www.dws.de/rentenfonds/lu2111935651-dws-invest-esg-euro-high-yield-lc/) Portfolio, from here on ESG-EHY. For the portfolio construction I reduced the number of equities in our portfolio to six of the biggest positions, assume low risk apetite and weigh the equities accordingl to Markowitz Efficient Frontier Theory. Lastly, I will look at performance metrics such as return and drawdown as well as volatility and sharpe ratio.

import libraries

In [ ]:
!pip install PyPortfolioOpt

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import math
import pypfopt

import yfinance as yf

fetch data

In [5]:
# list of equity tickers
portfolio = ["EDP.LS", "FRVIA.PA", "ISP.MI", "LHA.DE", "QTS.F", 'TIT.MI']

# Define the date range for historical data
start_date = "2018-01-01"
end_date = "2023-01-01"

df = yf.download(portfolio, start=start_date, end=end_date)['Close']
df.head()

[*********************100%%**********************]  6 of 6 completed


,EDP.LS,FRVIA.PA,ISP.MI,LHA.DE,QTS.F,TIT.MI
Date,,,,,,
2018-01-02,2.902,64.940002,2.754,30.280001,82.919998,0.7255
2018-01-03,2.944,66.040001,2.750,30.900000,82.349998,0.7250
2018-01-04,2.974,66.379997,2.822,30.719999,81.260002,0.7340
2018-01-05,2.994,68.580002,2.816,30.260000,80.790001,0.7385
2018-01-08,2.978,69.360001,2.808,30.559999,82.230003,0.7525


## Portfolio Construction

ref [Humboldt University Berlin](https://github.com/baerenstein/DEDA_class_SoSe2023/blob/main/DEDA_class_SoSe2023_PFM_using_Markowitz/PFM_using_Markowitz.ipynb)

In [7]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [8]:
# Calculate expected returns and sample covariance
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

# Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.save_weights_to_file("weights.csv")  # saves to file
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

OrderedDict([('EDP.LS', 0.31926), ('FRVIA.PA', 0.0), ('ISP.MI', 0.0), ('LHA.DE', 0.0), ('QTS.F', 0.68074), ('TIT.MI', 0.0)])
Expected annual return: 15.4%
Annual volatility: 26.1%
Sharpe Ratio: 0.51


(0.15373687079409432, 0.26101353318759074, 0.5123752364900462)

In [11]:
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

latest_prices = get_latest_prices(df)
weights = cleaned_weights

da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=10000)
allocation, leftover = da.greedy_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))

Discrete allocation: {'QTS.F': 35, 'EDP.LS': 685}
Funds remaining: $69.64
